1. Establish a connection between Python and the Sakila database.

In [2]:
# 📚 Basic libraries
import pandas as pd # data manipulation
import numpy as np # numerical operations
import warnings # warning messages management
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

# 🌐 New Libraries
from getpass import getpass # get password without showing it
import pymysql # MySQL connection
import requests # HTTP requests
import time # time operations

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

def create_connection(): # create a connection to a MySQL database
    """
    Create a connection to a MySQL database.
    Prompts the user for the database password and database name.
    
    Returns:
        cnx: pymysql.connections.Connection object
            The connection object to the MySQL database.
    """
    password = getpass("Please, kindly insert your password: ")
    database = input("Please, kindly insert your database name: ")
    cnx = pymysql.connect(user='root', password=password,
                          host='localhost', database=database)
    if cnx.open:
        print("Connection open")
    else:
        print("Connection is not successfully open")
    return cnx

In [5]:
connection=create_connection()

Connection open




2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	- `month`: an integer representing the month for which rental data is to be retrieved.
	- `year`: an integer representing the year for which rental data is to be retrieved.

	The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

	


In [14]:
def rentals_month(month, year):
    query = f"SELECT * FROM rental WHERE YEAR(rental_date)= {year} AND MONTH(rental_date)={month}";
    df=pd.read_sql(query,connection)
    return df


In [15]:
rentals_month(5,2005)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53



3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

	The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".


	*Hint: Consider making use of pandas [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)*

	


In [26]:
def rental_count_month (month,year):
    query=f"SELECT COUNT(r.rental_id) AS rentals_{month}_{year}, CONCAT(c.first_name, ' ', c.last_name) AS customer_name FROM rental r JOIN customer c USING (customer_id) WHERE YEAR(r.rental_date) = {year} AND MONTH(r.rental_date) = {month} GROUP BY customer_name ORDER BY rentals_{month}_{year} DESC;"
    df=pd.read_sql(query,connection)
    return df

In [29]:
rental_count_month(5,2005)

,rentals_5_2005,customer_name
0,8,SUE PETERS
1,7,EDNA WEST
2,7,LESLIE SEWARD
3,6,RUTH MARTINEZ
4,6,HEATHER MORRIS
...,...,...
515,1,ROSS GREY
516,1,ANDY VANHORN
517,1,SETH HANNON
518,1,TERRENCE GUNDERSON



4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 
The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [42]:
def compare_rentals(month1,year1,month2,year2):
    df1=rental_count_month(month1,year1)
    df2=rental_count_month(month2,year2)
    df1.rename(columns={"rental_count": f"rentals_{month1}_{year1}"}, inplace=True)
    df2.rename(columns={"rental_count": f"rentals_{month2}_{year2}"}, inplace=True)
    df = df1.merge(df2, on="customer_name", how="outer").fillna(0)
    df["difference"] = df[f"rentals_{month1}_{year1}"] - df[f"rentals_{month2}_{year2}"]
    return df

In [43]:
compare_rentals(5,2005,6,2005)

,rentals_5_2005,customer_name,rentals_6_2005,difference
0,4.0,AARON SELBY,3.0,1.0
1,2.0,ADAM GOOCH,1.0,1.0
2,1.0,ADRIAN CLARY,1.0,0.0
3,2.0,AGNES BISHOP,3.0,-1.0
4,1.0,ALAN KAHN,3.0,-2.0
...,...,...,...,...
593,3.0,WILLIE MARKHAM,4.0,-1.0
594,0.0,WILMA RICHARDS,4.0,-4.0
595,2.0,YOLANDA WEAVER,5.0,-3.0
596,2.0,YVONNE WATKINS,4.0,-2.0
